# Составление фондового индекса

Метод главных компонент (principal component analysis, PCA) — это один из методов обучения без учителя, который позволяет сформировать новые признаки, являющиеся линейными комбинациями старых. При этом новые признаки строятся так, чтобы сохранить как можно больше дисперсии в данных. Иными словами, метод главных компонент понижает размерность данных оптимальным с точки зрения сохранения дисперсии способом.

Основным параметром метода главных компонент является количество новых признаков. Как и в большинстве методов машинного обучения, нет четких рекомендаций по поводу выбора значения этого параметров. Один из подходов — выбирать минимальное число компонент, при котором объясняется не менее определенной доли дисперсии (это означает, что в выборке сохраняется данная доля от исходной дисперсии).

В этом задании понадобится измерять схожесть двух наборов величин. Если имеется набор пар измерений (например, одна пара — предсказания двух классификаторов для одного и того же объекта), то охарактеризовать их зависимость друг от друга можно с помощью корреляции Пирсона. Она принимает значения от -1 до 1 и показывает, насколько данные величины линейно зависимы. Если корреляция равна -1 или 1, то величины линейно выражаются друг через друга. Если она равна нулю, то линейная зависимость между величинами отсутствует.

**Данные**<br/>
В этом задании мы будем работать с данными о стоимостях акций 30 крупнейших компаний США. На основе этих данных можно оценить состояние экономики, например, с помощью индекса Доу-Джонса. Со временем состав компаний, по которым строится индекс, меняется. Для набора данных был взят период с 23.09.2013 по 18.03.2015, в котором набор компаний был фиксирован (подробнее почитать о составе можно по ссылке из материалов). 

Одним из существенных недостатков индекса Доу-Джонса является способ его вычисления — при подсчёте индекса цены входящих в него акций складываются, а потом делятся на поправочный коэффициент. В результате, даже если одна компания заметно меньше по капитализации, чем другая, но стоимость одной её акции выше, то она сильнее влияет на индекс. Даже большое процентное изменение цены относительно дешёвой акции может быть нивелировано незначительным в процентном отношении изменением цены более дорогой акции.

In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA


## 1. Загрузите данные close_prices.csv.

В этом файле приведены цены акций 30 компаний на закрытии торгов за каждый день периода.

In [3]:
df = pd.read_csv("data/close_prices.csv")
df.head()

,date,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,2013-09-23,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,2013-09-24,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,2013-09-25,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,2013-09-26,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,2013-09-27,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


In [7]:
X_train = df.drop(['date'], axis = 1)
X_train.head()

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,76.070000,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,75.959999,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


## 2. На загруженных данных обучите преобразование PCA с числом компоненты равным 10.

Скольких компонент хватит, чтобы объяснить 90% дисперсии?

In [25]:
pca = PCA(n_components=10)
pca.fit(X_train)



sum_var = 0
n = 0
for i in enumerate(pca.explained_variance_ratio_):
    sum_var += i[1]
    n += 1
    if sum_var >= 0.9: 
        break
n

4

In [26]:
outFile_1 = open('01_Zadanie.txt', 'w', encoding='utf8')

print(n, file=outFile_1)
outFile_1.close()

## 3. Примените построенное преобразование к исходным данным и возьмите значения первой компоненты.

In [50]:
data_pca = pd.DataFrame(pca.transform(X_train))[0]
data_pca

0     -50.902404
1     -52.846909
2     -54.614439
3     -52.600566
4     -52.370123
         ...    
369    81.290980
370    77.903534
371    83.798199
372    79.195789
373    82.968859
Name: 0, Length: 374, dtype: float64

## 4. Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv.
Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса?

In [43]:
df2 = pd.read_csv('data/djia_index.csv')
df2

,date,^DJI
0,2013-09-23,15401.379883
1,2013-09-24,15334.589844
2,2013-09-25,15273.259766
3,2013-09-26,15328.299805
4,2013-09-27,15258.240234
...,...,...
369,2015-03-12,17895.220703
370,2015-03-13,17749.310547
371,2015-03-16,17977.419922
372,2015-03-17,17849.080078


In [55]:
corr = np.corrcoef(data_pca, df2["^DJI"])
corr[1, 0]

0.9096522193050237

In [57]:
outFile_2 = open('02_Zadanie.txt', 'w', encoding='utf8')

print(f"{corr[1, 0]:.2f}", file=outFile_2)
outFile_2.close()

## 5. Какая компания имеет наибольший вес в первой компоненте?

Укажите ее название с большой буквы.

In [67]:
pca.components_[0]

array([ 1.61383840e-02,  1.20644923e-01, -5.16612711e-02,  5.04842369e-02,
       -1.25859933e-01,  1.14089567e-01,  2.33906290e-01, -6.20513749e-03,
        2.51227032e-01,  2.88996029e-01, -2.64998795e-01,  9.31320168e-02,
        9.13948403e-02,  4.69879340e-02,  2.90549417e-02, -2.61068828e-02,
        3.29615584e-01,  7.13897133e-02,  7.62295699e-02,  2.11888868e-01,
        2.30922941e-02,  7.77316954e-02, -7.20594590e-03,  1.89479745e-01,
        3.21564017e-01,  5.36834873e-02,  5.79683946e-01,  1.09122230e-04,
        8.71614334e-02, -4.29421420e-02])

In [68]:
print(X_train.columns[np.argmax(pca.components_[0])])

V


In [69]:
outFile_3 = open('03_Zadanie.txt', 'w', encoding='utf8')

print(X_train.columns[np.argmax(pca.components_[0])], file=outFile_3)
outFile_3.close()